
- Dodac spoznienia/przyjscia wczesniej w umawianym

- csv z danymi o id_Pacjenta, id_gabinetu, arrival_time, service_start_time, czas_oczekiwania, service_end_time, czas_

- procesowanie danych

*lekarze przerwa od pracy

- naprawić żeby klienci nie wchodzili do gabinetu {service_time} przed zamknięciem **chyba zrobione**

- dlugosci kolejki zapisywana

In [137]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import simpy
import seaborn
import matplotlib.pyplot as plt

In [138]:
class Pacjent:
    def __init__(self, id):
        self.id = id
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None


In [139]:
class Gabinet:
    def __init__(self, id, env):
        self.id = id
        self.env = env
        self.patients_served = 0

In [140]:
class Clinic:
    def __init__(self, env, number_of_rooms, service_time, lambda_per_hour=14/60, seed=None, sim_time=480):
        self.curr_patient_id = 1
        self.env = env
        self.service_time = service_time
        self.lambda_per_hour = lambda_per_hour
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1, env=self.env) for i in range(number_of_rooms)]
        self.rooms = simpy.Store(self.env, capacity=3)
        self.sim_time = sim_time
        for room in self.list_rooms:
            self.rooms.put(room)
        self.processed_patients = []

    def generate_patients(self):
        def time_between_new_patients():
            if self.seed:
                np.random.seed(self.seed)
            return np.random.exponential(1/self.lambda_per_hour)

        while True:
            patient = Pacjent(id=self.curr_patient_id)
            patient.arrival_time = self.env.now
            print(f"Czas {np.trunc(self.env.now)}: Pacjent {patient.id} przybył do kliniki")
            self.env.process(self.serve_patient(patient))
            self.curr_patient_id += 1
            yield self.env.timeout(time_between_new_patients())

    def serve_patient(self, patient): 
        room = yield self.rooms.get()
        if self.env.now >= self.sim_time-self.service_time: # Pacjent przychodzi za późno
            print(f"Czas {np.trunc(self.env.now)}: Pacjent {patient.id} przybył za późno do kliniki i odchodzi")
            yield self.rooms.put(room)
            return
        patient.service_start_time = self.env.now
        print(f"Czas {np.trunc(self.env.now)}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
        yield self.env.timeout(self.service_time)

        patient.service_end_time = self.env.now
        room.patients_served += 1
        print(f"Czas {np.trunc(self.env.now)}: Pacjent {patient.id} wychodzi z gabinetu {room.id}, czekal od {np.trunc(patient.arrival_time)} do {np.trunc(patient.service_start_time)}")
        self.processed_patients.append(patient)
        yield self.rooms.put(room)

    def run(self):
        self.env.process(self.generate_patients())
        env.run(until=self.sim_time)

    def stats(self):
        def patient_bar_plot():
            patients_served_ls = []
            id_ls = []
            for room in self.list_rooms:
                patients_served_ls.append(room.patients_served)
                id_ls.append(str(room.id))
            fig = seaborn.barplot(x=id_ls, y=patients_served_ls)
            fig.set_xlabel('Gabinet')
            fig.set_ylabel('Ilosc Pacjentow')
            plt.savefig("patients_served_noshow.png")
            plt.show()

        patient_bar_plot()

In [141]:
#bez umawiania
env = simpy.Environment()

In [142]:
clinic = Clinic(env, number_of_rooms=3, service_time=15)

In [143]:
clinic.run()

Czas 0: Pacjent 1 przybył do kliniki
Czas 0: Pacjent 1 wchodzi do gabinetu 1 
Czas 4.0: Pacjent 2 przybył do kliniki
Czas 4.0: Pacjent 2 wchodzi do gabinetu 2 
Czas 7.0: Pacjent 3 przybył do kliniki
Czas 7.0: Pacjent 3 wchodzi do gabinetu 3 
Czas 7.0: Pacjent 4 przybył do kliniki
Czas 10.0: Pacjent 5 przybył do kliniki
Czas 14.0: Pacjent 6 przybył do kliniki
Czas 15: Pacjent 1 wychodzi z gabinetu 1, czekal od 0 do 0
Czas 15: Pacjent 4 wchodzi do gabinetu 1 
Czas 16.0: Pacjent 7 przybył do kliniki
Czas 19.0: Pacjent 2 wychodzi z gabinetu 2, czekal od 4.0 do 4.0
Czas 19.0: Pacjent 5 wchodzi do gabinetu 2 
Czas 22.0: Pacjent 3 wychodzi z gabinetu 3, czekal od 7.0 do 7.0
Czas 22.0: Pacjent 6 wchodzi do gabinetu 3 
Czas 30: Pacjent 4 wychodzi z gabinetu 1, czekal od 7.0 do 15
Czas 30: Pacjent 7 wchodzi do gabinetu 1 
Czas 30.0: Pacjent 8 przybył do kliniki
Czas 33.0: Pacjent 9 przybył do kliniki
Czas 33.0: Pacjent 10 przybył do kliniki
Czas 33.0: Pacjent 11 przybył do kliniki
Czas 34.0: Pac

In [144]:
clinic.stats()